Instalando biblioteca

In [3]:
!pip install -q --upgrade langchain langchain_groq faiss-cpu sentence-transformers langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


Importando as ferramentas

In [4]:
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [5]:
# Pega a chave da Groq
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print("Chave da API da Groq carregada com sucesso.")
except Exception as e:
    print("Erro ao carregar a chave da Groq. Verifique se você a salvou corretamente nos 'Secrets' do Colab.")
    print(f"Detalhe do erro: {e}")


# Groq e Llama 3
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    groq_api_key=GROQ_API_KEY
)

# Ferramenta de Embeddings LOCAL
print("\nCarregando o modelo de embeddings local... (pode demorar na primeira vez)")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

print("\nAmbiente final configurado com Groq (LLM) e Embeddings Locais (Hugging Face)!")

Chave da API da Groq carregada com sucesso.

Carregando o modelo de embeddings local... (pode demorar na primeira vez)


/tmp/ipython-input-44161281.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Ambiente final configurado com Groq (LLM) e Embeddings Locais (Hugging Face)!


In [6]:
docs = []
pasta_de_arquivos = "/content/"

try:
    for arquivo in os.listdir(pasta_de_arquivos):
        if arquivo.endswith(".txt"):
            caminho_completo = os.path.join(pasta_de_arquivos, arquivo)
            loader = TextLoader(caminho_completo, encoding='utf-8')
            docs.extend(loader.load())

    if not docs:
        print("Nenhum arquivo .txt foi encontrado. Certifique-se de criá-los na pasta /content/.")
    else:
        print(f"Sucesso! Foram carregados {len(docs)} documentos.")
        print(f"Exemplo do primeiro documento carregado: {docs[0].metadata}")

except Exception as e:
    print(f"Ocorreu um erro ao carregar os arquivos: {e}")

Sucesso! Foram carregados 3 documentos.
Exemplo do primeiro documento carregado: {'source': '/content/Economia Política.txt'}


In [7]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_documents(docs)

print(f"O número original de documentos era: {len(docs)}")
print(f"Após o fatiamento, agora temos {len(chunks)} chunks.")

O número original de documentos era: 3
Após o fatiamento, agora temos 246 chunks.


In [8]:
print("Criando o banco de dados vetorial... Isso pode levar um momento.")
# Usa o modelo local de embeddings para transformar todos os chunks em vetores
# e armazená-los no FAISS.
vectorstore = FAISS.from_documents(chunks, embeddings)

# Cria o retriever que fará as buscas
retriever = vectorstore.as_retriever()

print("\nBanco de dados vetorial (FAISS) e retriever criados com sucesso!")

Criando o banco de dados vetorial... Isso pode levar um momento.

Banco de dados vetorial (FAISS) e retriever criados com sucesso!


In [9]:
# Template do Prompt
prompt = ChatPromptTemplate.from_template("""
Você é um assistente especializado em economia. Sua tarefa é responder às perguntas do usuário
baseando-se SOMENTE no contexto fornecido abaixo. Seja claro e direto.

Contexto:
{context}

Pergunta:
{input}
""")

# Cadeia de Documentos: Ensina o LLM a receber o contexto e a pergunta
document_chain = create_stuff_documents_chain(llm, prompt)

# Cadeia de Recuperação (RAG Chain):
# Orquestra todo o processo:
# - Pega a pergunta do usuário.
# - Usa o 'retriever' para buscar os chunks relevantes.
# - Passa os chunks e a pergunta para a 'document_chain' gerar a resposta.
rag_chain = create_retrieval_chain(retriever, document_chain)

print("Cadeia RAG completa criada com sucesso!")

Cadeia RAG completa criada com sucesso!


In [17]:
# Local para fazer perguntas
pergunta = "crie uma briga entre karl marx e adam smith" # pergunta baseada nas minhas anotacoes das disciplinas de economia do 2 periodo da UFRRJ

# Invoca a cadeia RAG com a sua pergunta
response = rag_chain.invoke({"input": pergunta})

# Imprime a resposta
print("--- Pergunta ---")
print(response["input"])
print("\n--- Resposta Gerada ---")
print(response["answer"])

--- Pergunta ---
crie uma briga entre karl marx e adam smith

--- Resposta Gerada ---
Que cena interessante!

(Karl Marx e Adam Smith se encontram em um café, cercados por pessoas que assistem à discussão com interesse)

Karl Marx: "Senhor Smith, você acha que a riqueza de uma nação reside apenas no trabalho anual de sua população? Isso é uma visão muito simplista e idealista. A verdadeira fonte da riqueza é a exploração do trabalho assalariado pelo capitalismo."

Adam Smith: "Ah, senhor Marx, você está confundindo as coisas. O trabalho é a verdadeira fonte de riqueza, mas não é a exploração que o faz valioso. É a produtividade e a eficiência do trabalhador que determinam o valor de seu trabalho. Além disso, o capitalismo é uma forma de distribuir a riqueza de forma justa e eficiente."

Karl Marx: "Justa e eficiente? Você está brincando, senhor Smith! O capitalismo é uma forma de exploração do trabalhador pelo capitalista. O trabalhador é forçado a vender sua força de trabalho para sob